In [ ]:
import pandas as pd

file_path = "marketing_AB.csv"
df = pd.read_csv(file_path)

display(df)

df.info()


,Unnamed: 0,user id,test group,converted,total ads,most ads day,most ads hour
0,0,1069124,ad,False,130,Monday,20
1,1,1119715,ad,False,93,Tuesday,22
2,2,1144181,ad,False,21,Tuesday,18
3,3,1435133,ad,False,355,Tuesday,10
4,4,1015700,ad,False,276,Friday,14
...,...,...,...,...,...,...,...
588096,588096,1278437,ad,False,1,Tuesday,23
588097,588097,1327975,ad,False,1,Tuesday,23
588098,588098,1038442,ad,False,3,Tuesday,23
588099,588099,1496395,ad,False,1,Tuesday,23


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 588101 entries, 0 to 588100
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   Unnamed: 0     588101 non-null  int64 
 1   user id        588101 non-null  int64 
 2   test group     588101 non-null  object
 3   converted      588101 non-null  bool  
 4   total ads      588101 non-null  int64 
 5   most ads day   588101 non-null  object
 6   most ads hour  588101 non-null  int64 
dtypes: bool(1), int64(4), object(2)
memory usage: 27.5+ MB


In [ ]:
import sqlite3

db_path = "marketing_AB.db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

create_table_query = """
CREATE TABLE IF NOT EXISTS marketing_AB (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_id INTEGER,
    test_group TEXT,
    converted BOOLEAN,
    total_ads INTEGER,
    most_ads_day TEXT,
    most_ads_hour INTEGER
);
"""

cursor.execute(create_table_query)
conn.commit()

df_sql = df.drop(columns=["Unnamed: 0"])
df_sql.columns = ["user_id", "test_group", "converted", "total_ads", "most_ads_day", "most_ads_hour"]

df_sql.to_sql("marketing_AB", conn, if_exists="append", index=False)



cursor.execute("SELECT * FROM marketing_AB;")
db_data = cursor.fetchall()


conn.close()


db_data

[(1, 1069124, 'ad', 0, 130, 'Monday', 20),
 (2, 1119715, 'ad', 0, 93, 'Tuesday', 22),
 (3, 1144181, 'ad', 0, 21, 'Tuesday', 18),
 (4, 1435133, 'ad', 0, 355, 'Tuesday', 10),
 (5, 1015700, 'ad', 0, 276, 'Friday', 14),
 (6, 1137664, 'ad', 0, 734, 'Saturday', 10),
 (7, 1116205, 'ad', 0, 264, 'Wednesday', 13),
 (8, 1496843, 'ad', 0, 17, 'Sunday', 18),
 (9, 1448851, 'ad', 0, 21, 'Tuesday', 19),
 (10, 1446284, 'ad', 0, 142, 'Monday', 14),
 (11, 1257223, 'ad', 0, 209, 'Wednesday', 11),
 (12, 1637531, 'ad', 0, 47, 'Wednesday', 13),
 (13, 1081965, 'ad', 0, 61, 'Tuesday', 20),
 (14, 1037215, 'ad', 0, 40, 'Friday', 13),
 (15, 1535652, 'ad', 0, 20, 'Tuesday', 19),
 (16, 1461774, 'ad', 1, 9, 'Wednesday', 18),
 (17, 1492276, 'ad', 0, 64, 'Tuesday', 12),
 (18, 1118924, 'ad', 0, 26, 'Wednesday', 11),
 (19, 900681, 'psa', 0, 248, 'Saturday', 19),
 (20, 1053783, 'ad', 0, 73, 'Friday', 19),
 (21, 1381767, 'ad', 0, 281, 'Wednesday', 11),
 (22, 1478526, 'ad', 0, 389, 'Saturday', 19),
 (23, 1436823, 'ad', 0,

In [ ]:
mysql_dump_path = "marketing_AB_mysql_dump.sql"

with sqlite3.connect(db_path) as conn:
    cursor = conn.cursor()

    cursor.execute("SELECT sql FROM sqlite_master WHERE type='table' AND name='marketing_AB';")
    sqlite_create_table = cursor.fetchone()[0]

    mysql_create_table = sqlite_create_table.replace("AUTOINCREMENT", "AUTO_INCREMENT").replace("BOOLEAN", "TINYINT(1)")

    cursor.execute("SELECT * FROM marketing_AB;")
    rows = cursor.fetchall()

    with open(mysql_dump_path, "w") as f:
        f.write("CREATE DATABASE IF NOT EXISTS marketing_AB;\n")
        f.write("USE marketing_AB;\n")
        f.write(mysql_create_table + ";\n")

        for row in rows:
            f.write(f"INSERT INTO marketing_AB VALUES {row};\n")

mysql_dump_path


'marketing_AB_mysql_dump.sql'